In [5]:
import trimesh
import numpy as np
import torch
import numpy as np
from scipy.spatial import cKDTree as KDTree
import pyrender
from PIL import Image
import os

In [2]:
def get_path(name, type='gt'):
    if type == 'gt':
        path = '/vision/hwjiang/datasets/Thingi10K/selected/data/{}/model/{}.obj'.format(name, name)
    elif type == 'nglod':
        path = '/vision/hwjiang/3d_generation/DeepLocalShapes/examples/nglod/{}/mc_res256.obj'.format(name)
    elif type == 'recon':
        path = '/vision/hwjiang/3d_generation/DeepLocalShapes/examples/all_computed_rot_quad_last/Reconstructions/18/Meshes/thingi/data/{}.ply'.format(name)
    return path

def load_mesh(path):
    mesh = trimesh.load(path)
    trimesh.repair.fix_inversion(mesh)
    rot_angle = np.radians(-90)
    rot_mat = trimesh.transformations.rotation_matrix(rot_angle, [1,0,0])
    mesh.apply_transform(rot_mat)
    return mesh

In [37]:
def compute_normalization_parameters(mesh, buffer=1.03):
    if not isinstance(mesh, trimesh.Trimesh):
        raise ValueError("Input must be a trimesh.Trimesh object.")

    # Only consider vertices used in the faces of the mesh
    used_vertices = mesh.vertices[mesh.faces.flatten()]
    # Compute min and max in each dimension
    min_coords = used_vertices.min(axis=0)
    max_coords = used_vertices.max(axis=0)
    # Compute the center
    center = (max_coords + min_coords) / 2.0
    # Make the mean zero
    distances = np.linalg.norm(used_vertices - center, axis=1)
    max_distance = distances.max()
    # Add some buffer
    max_distance *= buffer
    return -center, 1.0 / max_distance


def normalize_mesh(mesh, scale, offset):
    if not isinstance(mesh, trimesh.Trimesh):
        raise ValueError("Input must be a trimesh.Trimesh object.")
    # Create a copy of the mesh to avoid modifying the original mesh
    normalized_mesh = mesh.copy()
    # Normalize the vertices
    normalized_mesh.vertices = (normalized_mesh.vertices / scale) - offset
    return normalized_mesh

In [9]:
names = os.listdir('./examples/all_computed_rot_quad_last/Reconstructions/18/Meshes/thingi/data')
names = [it.replace('.ply', '') for it in names]
names

['372114',
 '35269',
 '78671',
 '47984',
 '72960',
 '72896',
 '68381',
 '72881',
 '95444',
 '68380',
 '92880',
 '75496',
 '87601',
 '441708',
 '77458',
 '135771',
 '551021',
 '87522',
 '73877']

In [10]:
for name in names:
    mesh_gt = load_mesh(get_path(name, type='gt'))
    center, scale = compute_normalization_parameters(mesh_gt)

    mesh_nglod = load_mesh(get_path(name, type='nglod'))

    mesh_recon = load_mesh(get_path(name, type='recon'))
    mesh_recon_norm = normalize_mesh(mesh_recon, scale, center)

    save_path = './examples/save/thingi/{}'.format(name)
    os.makedirs(save_path, exist_ok=True)

    with open(os.path.join(save_path, 'gt.obj'), 'w') as f:
        mesh_gt.export(f, file_type='obj')

    with open(os.path.join(save_path, 'nglod.obj'), 'w') as f:
        mesh_nglod.export(f, file_type='obj')

    with open(os.path.join(save_path, 'recon.obj'), 'w') as f:
        mesh_recon_norm.export(f, file_type='obj')

In [11]:
mesh_gt.vertices.min(), mesh_gt.vertices.max(), mesh_nglod.vertices.min(), mesh_nglod.vertices.max(), mesh_recon.vertices.min(), mesh_recon.vertices.max(), mesh_recon_norm.vertices.min(), mesh_recon_norm.vertices.max()

(-33.3679313659668,
 91.73355102539062,
 -33.26531388,
 91.73834745,
 -0.7944979667663574,
 0.7937690019607544,
 -33.020129125355176,
 91.74505896257571)

In [31]:
from scipy.spatial import cKDTree as KDTree

def compute_chamfer(points1, points2):
    # gen_points_kd_tree = KDTree(points2)
    # one_distances, one_vertex_ids = gen_points_kd_tree.query(points1)
    # gt_to_gen_chamfer = np.mean(np.square(one_distances))

    # other direction
    gt_points_kd_tree = KDTree(points1)
    two_distances, two_vertex_ids = gt_points_kd_tree.query(points2)
    gen_to_gt_chamfer = np.mean(np.square(two_distances))

    return gen_to_gt_chamfer * 2 #gt_to_gen_chamfer + gen_to_gt_chamfer

In [14]:
points_gt = trimesh.sample.sample_surface(mesh_gt, 30000)[0]
points_nglod = trimesh.sample.sample_surface(mesh_nglod, 30000)[0]
points_recon = trimesh.sample.sample_surface(mesh_recon_norm, 30000)[0]

In [15]:
points_gt.min(), points_gt.max(), points_nglod.min(), points_nglod.max(), points_recon.min(), points_recon.max()

(-33.28260584870858,
 91.72999014501119,
 -33.08811528834172,
 91.73371302968461,
 -32.905186568449004,
 91.73377526299664)

In [20]:
points_gt_norm = (points_gt + center) * scale
points_nglod_norm = (points_nglod + center) * scale
points_recon_norm = (points_recon + center ) * scale

In [27]:
compute_chamfer(points_gt_norm, points_recon_norm)

0.0018858830177252445

In [34]:
points_gt_norm.min(axis=0), points_gt_norm.max(axis=0), points_nglod_norm.min(axis=0), points_nglod_norm.max(axis=0), points_recon_norm.min(axis=0), points_recon_norm.max(axis=0)

(TrackedArray([-0.28822969, -0.90452258, -0.275304  ]),
 TrackedArray([0.28892364, 0.90428242, 0.27534764]),
 TrackedArray([-0.28932856, -0.90639004, -0.27630728]),
 TrackedArray([0.28693009, 0.90363156, 0.27504741]),
 TrackedArray([-0.29171609, -0.90613183, -0.27527705]),
 TrackedArray([0.28043954, 0.902576  , 0.2772757 ]))

In [38]:
dists_nglod, dists_recon = [], []

for name in names[::-1][:1]:
    mesh_gt = load_mesh(get_path(name, type='gt'))
    center, scale = compute_normalization_parameters(mesh_gt)

    mesh_nglod = load_mesh(get_path(name, type='nglod'))

    mesh_recon = load_mesh(get_path(name, type='recon'))
    mesh_recon_norm = normalize_mesh(mesh_recon, scale, center)

    points_gt = trimesh.sample.sample_surface(mesh_gt, 30000)[0]
    points_nglod = trimesh.sample.sample_surface(mesh_nglod, 30000)[0]
    points_recon = trimesh.sample.sample_surface(mesh_recon_norm, 30000)[0]
    points_gt_norm = (points_gt + center) * scale
    points_nglod_norm = (points_nglod + center) * scale
    points_recon_norm = (points_recon + center ) * scale

    dist1 = compute_chamfer(points_gt_norm, points_nglod_norm)
    dist2 = compute_chamfer(points_gt_norm, points_recon_norm)
    dists_nglod.append(dist1)
    dists_recon.append(dist2)
    print(name, dist1, dist2)

73877 0.00012383940501876686 0.0035566418794391787


In [33]:
np.mean(dists_nglod)

0.00010396154431624436